In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 圖片集
data_path = pathlib.Path("/content/drive/MyDrive/train200/")
# 辨識種類數目
category = 10

# 訓練、驗證資料集
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels="inferred",
    label_mode="int",
    image_size=(320,320),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=42
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels="inferred",
    label_mode="int",
    image_size=(320,320),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    seed=42
)

Found 2000 files belonging to 10 classes.
Using 1600 files for training.
Found 2000 files belonging to 10 classes.
Using 400 files for validation.


In [ ]:
# 照片數據增強
data_augmentation = keras.Sequential([
    layers.RandomZoom(0.2), # 圖片尺寸縮放
    layers.RandomContrast(factor=0.1), # 圖片對比度
    layers.RandomRotation(factor=0.2), # 圖片旋轉
    layers.GaussianNoise(0.2), # 增加圖片雜訊
])

In [ ]:
# 圖片輸入大小與通道 = 3
# 手動調整為 640, 640
inputs = keras.Input(shape=(320, 320, 3))  # 将输入形状改为(None, None, 3)

# 模型結構: VGG13 結構，不包含最後三層的Flatten神經層
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x) # 圖片預處理，將數值統一拉為[0, 1]
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(512, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(512, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(512, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(512, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.GlobalAveragePooling2D()(x)

# x = layers.Dense(128)(x)
x = layers.Dropout(0.5)(x) # 隨機選取50%的神經元做預測

outputs = layers.Dense(category, activation="softmax")(x)
model = keras.Model(inputs, outputs)
# 模型優化，使用Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.000157)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"])

In [ ]:
#model.summary()

In [ ]:
# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
              patience=5,
              mode="min")

# ModelCheckpoint
checkpoint = ModelCheckpoint('vgg13.{epoch:03d}-{val_loss:.4f}.pb',
                monitor='val_loss',
                verbose=0,
                save_best_only=True,
                mode="min")

callbacks = [early_stop, checkpoint]

In [ ]:
# # 保留較好的訓練權重
# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         filepath='chi102_{epoch:02d}_{val_loss:4f}.pb',
#         save_best_only = True,
#         monitor = "val_loss",
#         verbose = 0
#     )
# ]
# 訓練結果
history = model.fit(
    train_dataset,
    #validation_split=0.2,
    epochs=200,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/200
50/50 [==============================] - 48s 842ms/step - loss: 2.3045 - acc: 0.0950 - val_loss: 2.3028 - val_acc: 0.0875
Epoch 2/200
50/50 [==============================] - 40s 750ms/step - loss: 2.3032 - acc: 0.1019 - val_loss: 2.3030 - val_acc: 0.0850
Epoch 3/200
50/50 [==============================] - 40s 753ms/step - loss: 2.3027 - acc: 0.1044 - val_loss: 2.3030 - val_acc: 0.0850
Epoch 4/200
50/50 [==============================] - 39s 744ms/step - loss: 2.3023 - acc: 0.1019 - val_loss: 2.3032 - val_acc: 0.1175
Epoch 5/200
50/50 [==============================] - 39s 745ms/step - loss: 2.3026 - acc: 0.1000 - val_loss: 2.3030 - val_acc: 0.0850
Epoch 6/200
50/50 [==============================] - 42s 800ms/step - loss: 2.3020 - acc: 0.1006 - val_loss: 2.2956 - val_acc: 0.0850
Epoch 7/200
50/50 [==============================] - 43s 804ms/step - loss: 2.1612 - acc: 0.1494 - val_loss: 2.1730 - val_acc: 0.1725
Epoch 8/200
24/50 [=============>................] - ETA: 16s 

In [ ]:
# 繪製圖形
# Accuracy 圖形
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()

# Loss 圖形
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

# json_config = model.to_json()
# new_model = keras.models.model_from_json(json_config)
# new_model.save("mmmmodel_weights.h5")
# model.save('8888my_model', save_format="json")

# Accuracy 與 Loss 結合

plt.plot(epochs, acc, linestyle='--', color='blue')
plt.plot(epochs, val_acc, color='red')
plt.plot(epochs, loss, linestyle='--', color='blue')
plt.plot(epochs, val_loss, color='red')
plt.title('Accuracy and Loss for Training and Validation')
plt.ylabel('Accuracy and Loss')
plt.xlabel('epoch')
#plt.xticks(epochs)
plt.legend(['acc','val_acc', 'loss', 'val_loss'], loc = 'upper left')
plt.show()